In [1]:
pip install spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
 
with open('/kaggle/input/d/lalithmovva/legal-ner/NER_TRAIN_PREAMBLE.json', 'r') as f:
    data = json.load(f)

In [3]:
import json
with open('/kaggle/input/d/lalithmovva/legal-ner/NER_TRAIN_JUDGEMENT.json', 'r') as f:
    data1 = json.load(f)

In [4]:
training_data = {'classes' : [], 'annotations' : []}
for example in data:
  
  if len(example['annotations']) >= 1 and len(example['annotations'][0]['result']) >= 1:
    temp_dict = {}
    temp_dict['text'] = example['data']['text']
    temp_dict['entities'] = []
    start = example['annotations'][0]['result'][0]['value']['start']
    end = example['annotations'][0]['result'][0]['value']['end']
    label = example['annotations'][0]['result'][0]['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
        
  
print(training_data['annotations'][0])

{'text': "In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n", 'entities': [(7, 40, 'COURT')]}


In [5]:
for example in data1:
  
  if len(example['annotations']) >= 1 and len(example['annotations'][0]['result']) >= 1:
    temp_dict = {}
    temp_dict['text'] = example['data']['text']
    temp_dict['entities'] = []
    start = example['annotations'][0]['result'][0]['value']['start']
    end = example['annotations'][0]['result'][0]['value']['end']
    label = example['annotations'][0]['result'][0]['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)

In [6]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [7]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 8816/8816 [00:06<00:00, 1383.64it/s]


In [8]:
!python -m spacy init fill-config /kaggle/input/d/lalithmovva/legal-ner/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-13 02:42:08,501] [INFO] Set up nlp object from config
[2022-12-13 02:42:08,512] [INFO] Pipeline: ['transformer', 'ner']
[2022-12-13 02:42:08,517] [INFO] Created vocabulary
[2022-12-13 02:42:08,520] [INFO] Finished initializing nlp object
Downloading: 100%|███████████████████████████| 29.0/29.0 [00:00<00:00, 33.0kB/s]
Downloading: 100%|██████████████████████████████| 411/411 [00:00<00:00, 604kB/s]
Downloading: 100%|███████████████████████████| 208k/208k [00:00<00:00, 2.74MB/s]
Downloading: 100%|███████████████████████████| 426k/426k [00:00<00:00, 4.49MB/s]
Downloading: 100%|███████████████████████████| 251M/251M [00:04<00:00, 62.2MB/s]
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_la

In [10]:
import json
 
with open('/kaggle/input/legal-ner1/NER_DEV_PREAMBLE.json', 'r') as f:
    dev_data1 = json.load(f)

In [11]:
import json
 
with open('/kaggle/input/legal-ner1/NER_DEV_PREAMBLE.json', 'r') as f:
    dev_data2 = json.load(f)

In [12]:
nlp_ner = spacy.load("model-best")


doc = nlp_ner(dev_data1[0]['data']['text'])
colors = {"COURT": "#F67DE3", 
          "JUDGE": "#7DF6D9", 
          "PETITIONER": "#FFFFFF",
          'RESPONDENT': '#FF0000',
          'LAWYER': '#F08080',
          'JUDGE': '#F9E79F',
          'DATE' :'#CA2E0D',
          'ORG' : '#AEB815',
          'GPE' :'#77AA12',
          'STATUTE' :'#219872',
          'PROVISION' :'#154E49',
          'PRECEDENT' :'#134563',
          'CASE_NUMBER' :'#8A2BA9',
          'WITNESS' :'#BC1E8A',
          'OTHER_PERSON' :'#BC1E38', 
         }
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [14]:
correct_pred_cnt = 0
total_cnt = 0

for data in dev_data1:
    
    total_cnt += 1
    
    model_input_text = data['data']['text']
    model_pred = nlp_ner(model_input_text)
    
    pred_label_list = []
    for ent in model_pred.ents:
        pred_label_list.append(ent.label_)
        
    label_list = []
    for label in data['annotations'][0]['result']:
        label_list.append(label['value']['labels'][0])
          
    if pred_label_list == label_list:
        correct_pred_cnt += 1

accuracy = (correct_pred_cnt / total_cnt) * 100


In [16]:
print(accuracy) 

83.62447278114793
